In [51]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
import numpy as np
import csv

In [26]:
train = pd.read_csv('train_yelp_60k.csv')
test = pd.read_csv('test_yelp_60k.csv')

In [27]:
# Drop ID from test and train dataframe
train_df = train.drop('ID', 1)
test_df = test.drop('ID', 1)

<ipython-input-27-2b974a0e1338>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_df = train.drop('ID', 1)
<ipython-input-27-2b974a0e1338>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_df = test.drop('ID', 1)


In [28]:
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
def preprocess(sentence):

  sentence = sentence.lower()
  sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)

  tokens = sentence.split()
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

  stop_words = set(stopwords.words('english'))
  lemmatized_tokens = [word for word in lemmatized_tokens if word not in stop_words]

  return ' '.join(lemmatized_tokens)

In [30]:
# apply same preprocessing to train and test dataframes
train_df['Text'] = [preprocess(document) for document in train_df['Text']]
test_df['Text'] = [preprocess(document) for document in train_df['Text']]

In [31]:
vectorizer = TfidfVectorizer(max_features=5000)
vectorizer.fit(train_df['Text'])
X = vectorizer.transform(train_df['Text'])
X_test = vectorizer.transform(test_df['Text'])

In [32]:
columns=vectorizer.get_feature_names_out()

In [33]:
vectorizer_train_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
vectorizer_test_df = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())

In [34]:
vectorizer_df
X_train = vectorizer_train_df
y_train = train_df['Class']
X_test = vectorizer_test_df

In [35]:
model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [52]:
test = pd.read_csv('test_yelp_60k.csv')
ids = test['ID'].to_numpy()
data = np.column_stack((ids, y_pred))

csv_file_path = 'prediction1.csv'

# Write to CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['ID', 'CLASS'])
    csv_writer.writerows(data)